In [1]:
# -*- coding: utf-8 -*-
import socket

def main():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("localhost", 50007))    # 指定したホスト(IP)とポートをソケットに設定

    s.listen(2)                     # 2つの接続要求を待つ
    
    while(1):
        soc1, addr1 = s.accept()          # 要求が来るまでブロック
        print("1st: Conneted by"+str(addr1))  #サーバ側の合図

        soc2, addr2 = s.accept()          # 要求が来るまでブロック
        print("2nd: Conneted by"+str(addr2))  #サーバ側の合図

        data = bytes('ちゃっとすたーと', encoding='utf-8')
        while (1):
            
            soc1.send(data)              # ソケットにデータを送信
            data = soc1.recv(1024)       # データを受信（1024バイトまで）
            print("Client1>",data.decode('utf-8'))       # サーバー側の書き込みを表示
            if data == b"q":             # qが押されたら終了
                soc2.send(b'Client1 quited')              # ソケットにデータを送信
                break

            soc2.send(data)              # ソケットにデータを送信
            data = soc2.recv(1024)       # データを受信（1024バイトまで）
            print("Client2>",data.decode('utf-8'))       # サーバー側の書き込みを表示
            if data == b'q':             # qが押されたら終了
                soc1.send(b'Client2 quited')              # ソケットにデータを送信
                break
        
        soc1.close()
        soc2.close()
        data = input("Server>")
        if data == "q": break
        

In [ ]:
main()

複数同時

In [1]:
import socket
import threading

HOST = 'localhost'
PORT = 9998
clients = []

def remove_conection(con, address, client_id):
    """クライアントと接続を切る"""

    print('[切断: Client{:2}]{}'.format(client_id, address))
    con.close()
    clients.remove((con, address, client_id))


def server_start():
    """サーバをスタートする"""
 
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind((HOST, PORT))
    sock.listen(10)

    client_id_counter = 0
    while True:
        con, address = sock.accept()
        
        # 通常接続かシャットダウン要求かの判定
        data = con.recv(1024)
        if b'server shutdown' in data:
            if clients != [] and b'force' not in data:
                con.sendto(b'reject', address)
                con.close()
                print('rejected shutdown request')
                continue
            else:
                con.sendto(b'accept', address)
                con.close()
                print('accepted shutdown request')
                break
        
        print("[接続: Client{:2}]{}".format(client_id_counter, address))
        clients.append((con, address, client_id_counter))
        handle_thread = threading.Thread(target=handler,
                                         args=(con, address, client_id_counter),
                                         daemon=True)
        handle_thread.start()

        client_id_counter += 1
        
    # jupyter のせいか、 deamon が終了しないので手動で終了
    for c in clients:
        remove_conection(*c)
        
    #return 'server shutdown'

def handler(con, address, client_id):
    """クライアントからデータを受信する"""
 
    try:
        while True:
            try:
                data = con.recv(1024)
            except ConnectionResetError:
                remove_conection(con, address, client_id)
                break
            else:
                if not data:
                    remove_conection(con, address, client_id)
                    break
                else:
                    #data_for_print = data.decode("utf-8")
                    #if len(data_for_print) > 100:
                    #    data_for_print = data[:100] + '...'
                    #print("[受信: Client{:2}]{} - {}".format(client_id, address, data_for_print))
                    print("[受信: Client{:2}]{} - {}".format(client_id, address, str(data)))
                    for c in clients:
                        #c[0].sendto(data, c[1])
                        # NEW dataformat
                        # [client_id]c[data]
                        # e.g. from ID 24, data is 'Hello' then
                        # 24cHello
                        c[0].sendto(bytes(str(client_id),encoding='utf-8')+b'c'+data, c[1])
                    
    except ConnectionAbortedError: # クライアントと接続中にサーバー終了要求が来た場合、終了する
        pass
    
if __name__ == "__main__":
    #server_start()
    server_start()

[接続: Client 0]('127.0.0.1', 50976)
[接続: Client 1]('127.0.0.1', 51030)
[受信: Client 0]('127.0.0.1', 50976) - b'\xe3\x83\x86\xe3\x82\xb9\xe3\x83\x88'
[受信: Client 1]('127.0.0.1', 51030) - b'\xe3\x81\xa6\xef\xbd\x93\xe3\x81\xa4\xe3\x81\x8a'
[切断: Client 0]('127.0.0.1', 50976)
accepted shutdown request
[切断: Client 1]('127.0.0.1', 51030)


In [1]:
1

1

'30jd\x00 asjl'